In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 


In [2]:
df=pd.read_csv('data_test_mktg_anlst.csv',sep=';') # download
df.head()

,cust_id,source,orders,ltv
0,859,?utm_source=google&utm_medium=cpc&utm_campaign...,28,913
1,232,?utm_source=google&utm_medium=cpc&utm_campaign...,18,3157
2,2523,?utm_source=mytarget&utm_medium=cpc&utm_campai...,1,1208
3,4199,?utm_source=facebook&utm_medium=cpc&utm_campai...,36,-275
4,4069,?utm_source=facebook&utm_medium=cpc&utm_campai...,43,-210


# 1.  For each line, it is required to isolate the source, medium, campaign of the user


In [3]:
# divide into columns
new_df = df['source'].str.split('&',expand=True)
new_df.columns=['source','medium','campaign']
for column in new_df.columns:
    new_df[column]=new_df[column].str.replace(r'\w+\=','')

#remove sign "?"
new_df['source']=new_df['source'].str.replace(r'?','')

#Remove to age
new_df['campaign']=new_df['campaign'].str.replace(r'\w+age','')

#combine in the final table
final_df=pd.concat([df,new_df],axis=1)
final_df.columns=['cust_id','source1','orders','ltv','source','medium','campaign']
final_df.drop('source1', inplace=True, axis=1)
final_df.head()

<ipython-input-3-8f326937d6b2>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df[column]=new_df[column].str.replace(r'\w+\=','')
<ipython-input-3-8f326937d6b2>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  new_df['source']=new_df['source'].str.replace(r'?','')
<ipython-input-3-8f326937d6b2>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['campaign']=new_df['campaign'].str.replace(r'\w+age','')


,cust_id,orders,ltv,source,medium,campaign
0,859,28,913,google,cpc,25-34
1,232,18,3157,google,cpc,18-24
2,2523,1,1208,mytarget,cpc,25-34
3,4199,36,-275,facebook,cpc,25-34
4,4069,43,-210,facebook,cpc,25-34


In [4]:
# Check for uniqueness of "medium" values
final_df.medium.unique()

array(['cpc'], dtype=object)

In [5]:
# Check for uniqueness of "source" values
final_df.source.unique()

array(['google', 'mytarget', 'facebook'], dtype=object)

In [6]:
final_df.describe()

,cust_id,orders,ltv
count,5000.000000,5000.000000,5000.000000
mean,2500.500000,30.577800,405.193400
std,1443.520003,17.656441,1133.211199
min,1.000000,1.000000,-2479.000000
25%,1250.750000,16.000000,-493.500000
50%,2500.500000,30.000000,403.500000
75%,3750.250000,45.000000,1327.000000
max,5000.000000,64.000000,3240.000000


In [7]:
final_df.head()

,cust_id,orders,ltv,source,medium,campaign
0,859,28,913,google,cpc,25-34
1,232,18,3157,google,cpc,18-24
2,2523,1,1208,mytarget,cpc,25-34
3,4199,36,-275,facebook,cpc,25-34
4,4069,43,-210,facebook,cpc,25-34


# 2. Calculate the average LTV per user, and compare the average number of orders per channel (source). Are there significant differences?


In [8]:
# Determine if strings are unique 
print(len(final_df['cust_id'].unique()))
print(final_df.shape)

5000
(5000, 6)


In [9]:
print('average LTV per user=',final_df['ltv'].mean())
print('average number of orders=',final_df['orders'].mean())


average LTV per user= 405.1934
average number of orders= 30.5778


In [10]:
new=pd.DataFrame(final_df.groupby(['source'])[['orders','ltv']].mean()).rename(columns={'orders':'orders_mean','ltv':'ltv_mean'})
new['orders_differences_%']=(new['orders_mean']/final_df['orders'].mean()-1)*100
new['ltv_differences_%']=(new['ltv_mean']/final_df['ltv'].mean()-1)*100
new['cust_id_count']=final_df.groupby(['source']).cust_id.count()
new

,orders_mean,ltv_mean,orders_differences_%,ltv_differences_%,cust_id_count
source,,,,,
facebook,25.516322,359.524983,-16.552785,-11.270770,1501
google,32.303000,463.286000,5.642002,14.337005,1000
mytarget,32.927571,409.377351,7.684565,1.032581,2499


differences in:
 
1) by **orders** you can see that the facebook channel sags heavily by 16.6%
 
2) **LTV** shows that the facebook channel sags heavily 11.3%


# 3. Which channel do you think works best?

Efficiency can be considered according to different criteria:

In [11]:
new_effect=pd.DataFrame(final_df.groupby(['source'])[['orders','ltv',]].sum()).rename(columns={'orders':'orders_sum','ltv':'ltv_sum'})
new_effect['cust_id_count']=final_df.groupby(['source']).cust_id.count()
new_effect

,orders_sum,ltv_sum,cust_id_count
source,,,
facebook,38300,539647,1501
google,32303,463286,1000
mytarget,82286,1023034,2499


1)  LTV

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
labels=new_effect.index
patches, texts, pcts = ax.pie(
    new_effect['ltv_sum'], labels=labels, autopct='%.1f%%',
    wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'},
    textprops={'size': 'x-large'},
    startangle=90)
# For each wedge, set the corresponding text label color to the wedge's
# face color.
for i, patch in enumerate(patches):
  texts[i].set_color(patch.get_facecolor())
plt.setp(pcts, color='white')
plt.setp(texts, fontweight=600)
ax.set_title('lifetime value by channel', fontsize=18)
plt.tight_layout()

2)  orders

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
labels=new_effect.index
patches, texts, pcts = ax.pie(
    new_effect['orders_sum'], labels=labels, autopct='%.1f%%',
    wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'},
    textprops={'size': 'x-large'},
    startangle=90)
# For each wedge, set the corresponding text label color to the wedge's
# face color.
for i, patch in enumerate(patches):
  texts[i].set_color(patch.get_facecolor())
plt.setp(pcts, color='white')
plt.setp(texts, fontweight=600)
ax.set_title('Orders by channel', fontsize=18)
plt.tight_layout()

3) by the number of clients attracted

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
labels=new_effect.index
patches, texts, pcts = ax.pie(
    new_effect['cust_id_count'], labels=labels, autopct='%.1f%%',
    wedgeprops={'linewidth': 3.0, 'edgecolor': 'white'},
    textprops={'size': 'x-large'},
    startangle=90)
# For each wedge, set the corresponding text label color to the wedge's
# face color.
for i, patch in enumerate(patches):
  texts[i].set_color(patch.get_facecolor())
plt.setp(pcts, color='white')
plt.setp(texts, fontweight=600)
ax.set_title('Number of attracted customers by channels', fontsize=18)
plt.tight_layout()

# 4.  Is it possible to optimize each of the channels?


In [ ]:
new=pd.DataFrame(final_df.groupby(['source'])[['orders','ltv']].mean()).rename(columns={'orders':'orders_mean','ltv':'ltv_mean'})
new['orders_differences_%']=(new['orders_mean']/final_df['orders'].mean()-1)*100
new['ltv_differences_%']=(new['ltv_mean']/final_df['ltv'].mean()-1)*100
new['cust_id_count']=final_df.groupby(['source']).cust_id.count()


In [ ]:
new_opt=pd.DataFrame(final_df.groupby(['source','campaign'])[['ltv','orders']].mean())
new_opt['count_cust']=final_df.groupby(['source','campaign']).cust_id.count()
new_opt

In [ ]:
new

A) Profit optimization
1) **facebook** The number of customers through facebook comes 50% more than through google, but the number of orders and profits are much lower. This shows that there are a lot of customers, but they make a small number of orders. Therefore, as an optimization, it is necessary to work towards increasing or adjusting offers for the target audience (age 25-34) to increase LTV.

2) **google** The number of customers from google is the smallest, but the average LTV from google is the largest, therefore, you need to increase the number of customers coming through this channel.

3) **mytarget** The number of clients in the 25-34 age group is the largest, but LTV is average. Therefore, you need to increase or adjust offers for the target audience (age 25-34) to increase LTV.

**Final output:**

A) For 2/3 of the channels, it is necessary to work out / rework proposals, with an emphasis on the target audience. In addition, you need to intensify the development of the google channel. it is the most profitable.

One of the options for increasing efficiency is a detailed segmentation of the target audience, where an offer is made for each segment of the target audience.

B) Optimize costs.

Dividing into two groups by LTV (positive/negative), it can be seen that the same number of orders were made in these groups.

This indicates that about 40% of people bring a loss to the company (negative LTV) and it is worth paying attention to. Example, a person ordered 5 buns for 100 rubles, but the company paid 200 rubles for attracting this client. You need to analyze the basket of such customers, perhaps enter the minimum order amount.

In [ ]:
final_df[final_df.ltv<0].cust_id.count()/final_df.cust_id.count()

In [ ]:
pd.DataFrame(final_df[final_df.ltv>0].groupby(['campaign','source']).orders.mean())

In [ ]:
pd.DataFrame(final_df[final_df.ltv<0].groupby(['campaign','source']).orders.mean())

In [ ]:
final_df[final_df.ltv>0].groupby('campaign').cust_id.count()

You can also:

2) Increase the average check through incentive promotions, upselling and cross-selling technologies.

3) Increase the frequency of purchases, for example, through the use of email channels, push notifications and retargeting.

In [ ]:
new1=pd.DataFrame(final_df.groupby(['source','campaign'])[['orders','ltv']].sum())
new1


# Suggest and calculate 2-3 more indicators that characterize the quality of the groups of the attracted audience.

### 1) Group profitability

In [ ]:
pd.DataFrame(final_df.groupby('campaign').ltv.sum()).rename(columns={'ltv':'sum_ltv'})

### 2) Number of clients by groups

In [ ]:
pd.DataFrame(final_df.groupby('campaign').cust_id.count()).rename(columns={'cust_id':'count_cust'})

### 3) Median check

In [ ]:
pd.DataFrame(final_df.groupby('campaign').ltv.median()).rename(columns={'ltv':'ltv_median'})

### 4) Share of profitable clients

In [ ]:
final_df_profit=final_df[final_df.ltv<0].groupby('campaign').cust_id.count()
final_df_no_profit=final_df[final_df.ltv>0].groupby('campaign').cust_id.count()
pd.DataFrame(final_df_profit/final_df_no_profit).rename(columns={'cust_id':'proportion'})

**Group 18-24**

has a high indicator: in terms of the share of clients that make a profit

low score: median check.

**Group 25-34**

has a high indicator: the number of customers and the profitability of the group

low indicator: in terms of the share of customers that make a profit.

**Group 35+**

has a high indicator: median check

low score: number of clients and group profitability